# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('brown')
from nltk.corpus import brown
from sklearn.metrics.pairwise import pairwise_distances

import random
from datetime import datetime
import math
import json
import glob

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


# Utils

In [4]:
def build_vocabulary(corpus, n_vocab):

  idx_sents = []
  idx_count = 1


  word2idx = {'<UNK>': 0}
  word_counts = {}

  sentences = []

  for sent in corpus:
    for tok in sent:
      tok = tok.lower()
      if tok not in word_counts:
        word_counts[tok] = 1

      else: word_counts[tok] += 1

  n_vocab = min(n_vocab, len(word_counts))
  word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
  top_words = [w for w, count in word_counts[:n_vocab-1]] + [0]

  for sent in corpus:
    for tok in sent:
      if tok in top_words:
        if tok not in word2idx:
          word2idx[tok] = idx_count
          idx_count += 1

    sentence = [word2idx[token] if token in word2idx else 0 for token in sent]
    sentences.append(sentence)

  return sentences, word2idx


def find_analogies( w1, w2, w3, concat=True, we_file ='w2v_model.npz', w2i_file='w2v_word2idx.json'):
  npz = np.load(we_file)
  W1 = npz['arr_0']
  W2 = npz['arr_1']

  with open(w2i_file) as f:
    word2idx = json.load(f)
  
  idx2word = {y:x for x,y in word2idx.items()}

  vocab_sz = len(word2idx)

  if concat:
    We = np.hstack([W1, W2.T])

    print("We shape: ", We.shape)
    assert(vocab_sz == We.shape[0])

  else:
    We = (W1 + W2.T)/2
  
  V, D = We.shape

  king = We[word2idx[w1]]
  man = We[word2idx[w2]]
  woman = We[word2idx[w3]]
  v0 = king - man + woman

 
  distances = pairwise_distances(v0.reshape(1, D), We, metric=dist).reshape(V)
  idx = distances.argsort()[:4]
  best_idx = -1
  keep_out = [word2idx[w] for w in (w1, w2, w3)]
  for i in idx:
      if i not in keep_out:
          best_idx = i
          break
  best_word = idx2word[best_idx]

  print(w1, "-", w2, "=", best_word, "-", w3)


def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def plot_cost(self, cost, title):
  plt.plot(cost)
  plt.title(title)
  plt.show()

# Word2Vec Model

In [ ]:
class Word2Vec():

  def __init__(self, embedding_sz, vocab_sz, context_window = 3):
    self.embedding_sz = embedding_sz
    self.vocab_sz = vocab_sz
    self.context_window = context_window
     
    # The Embedding Layer (center word)
    self.W1 = np.random.rand(vocab_sz, embedding_sz)

    # The Dense Layer (context words)
    self.W2 = np.random.rand(embedding_sz, vocab_sz)

  
  def _get_neg_prob(self, corpus):
    self.neg_prob = np.zeros(self.vocab_sz)
    word_counts = {}
    word_sum = sum(len(sent) for sent in corpus)

    for sent in corpus:
      for word in sent:
        if word not in word_counts:
          word_counts[word] = 1
        
        else: 
          word_counts[word] += 1
    
    for i in range(self.vocab_sz):
      # 0.75 is a hyperparameter that was found to work well after hyperparameter tunining. 
      # This reduces the probability of picking extemely frequent words relative to less frequent ones
      self.neg_prob[i] = (word_counts[i] / word_sum)**0.75
    
    # asserting that we have as many word counts as the size of neg_prob
    assert(all(self.neg_prob >0))
    
    return self.neg_prob

  
  def _get_neg_samples(self, context, num_neg_samples):
    
    temp = {}
    for context_idx in context:
      temp[context_idx] = self.neg_prob[context_idx]
      self.neg_prob[context_idx] = 0

    neg_samples = np.random.choice(
        self.vocab_sz,
        size = num_neg_samples,
        replace = False,
        p = self.neg_prob/np.sum(self.neg_prob)
    )

    for context_idx in temp:
      self.neg_prob[context_idx] = temp[context_idx]
    
    return neg_samples
  
  def _get_prob_vector(self,center_word, context,isNeg = False):
    # multiplying the context and center_word
    A = center_word.dot(self.W2[:,context])
    if isNeg == False:
      # vect of dim (context,) => used for the objective function that is maximized when the probability is closest to 1
      prob_vector = sigmoid(A)
    else: 
      # vect of dim (context,) => used for the objective function that is maximized when the probability is closest to 0
      prob_vector = sigmoid(-A)

    return prob_vector
  
  def backprop(self, p, q, center_word, center_word_idx, context, neg_samples, dW1, dW2, lr):

    # no need to update the whole ebedding matrix as we are using negative sampling. just updating the context and negative samples
  
    dW2[:, context] = np.outer(center_word, p - 1)
    # 1-q = center_word.dot(self.W2[:,neg_samples])
    dW2[:, neg_samples] = np.outer(center_word, 1-q)

    self.W2[:, context] -= lr*dW2[:, context]
    self.W2[:, neg_samples] -= lr*dW2[:, neg_samples]

    dW1 = (p - 1).dot(self.W2[:,context].T) + (1 - q).dot(self.W2[:,neg_samples].T)
          
    self.W1[center_word_idx,:] -= lr*dW1
  

  def fit(self, corpus, num_neg_samples = 10, lr = 3e-5, epochs = 10):
    num_samples = len(corpus)
    vocab_sz = self.vocab_sz
    embedding_sz = self.embedding_sz
    self._get_neg_prob(corpus)

    cost_per_epoch = []
    samples_idx = list(range(num_samples))

    dW1 = np.zeros(self.W1.shape)
    dW2 = np.zeros(self.W2.shape)

    for i in range(epochs):
      t0 = datetime.now()
      random.shuffle(samples_idx)
      costs_per_epoch = []
      glob_count = 0

      for sent_idx in range(num_samples):
        glob_count += 1

        if glob_count % 5000 == 0:
          print(glob_count, " sentences done")

        j = samples_idx[sent_idx]
        sent = corpus[sent_idx]

        if len(sent) < 2 * self.context_window + 1:
          continue
      
        costs_per_sent = []
        num_words = len(sent)

        for w_idx in range(num_words):

          # center word : X*T.W1 where X is a one hot encoded vector with a 1 at the vocabulary index
          center_word = self.W1[sent[w_idx], :]

          start = max(0, w_idx - self.context_window)
          end = min(num_words, w_idx + 1 + self.context_window)

          context = np.concatenate((sent[start:w_idx], sent[w_idx + 1:end]))
          context = np.array(list(set(context)), dtype = np.int32)

          # getting the negative samples
          neg_samples = self._get_neg_samples(context, num_neg_samples)

          p = self._get_prob_vector(center_word,context, isNeg = False)
          q = self._get_prob_vector(center_word,neg_samples, isNeg = True)
          # cost function : negative log loss

          cost = - (np.log(q).sum() + np.log(p).sum())
          costs_per_sent.append(cost/(num_neg_samples + len(context)))

          self.backprop(p, q, center_word, sent[w_idx], context, neg_samples, dW1, dW2, lr)

        costs_per_epoch.append(np.mean(costs_per_sent))

      cost_per_epoch.append(np.mean(costs_per_epoch))
      print ( "time to complete epoch %d:" % i,  (datetime.now()-t0), "cost:", cost_per_epoch[-1])

      plot_cost(costs_per_epoch,'Numpy costs')

    plot_cost(cost_per_epoch, 'Numpy cost at each epoch')


  def save(self, fn):
    arrays = [self.W1, self.W2]
    np.savez(fn, *arrays)


def run():
  sentences, word2idx = build_vocabulary(brown.sents(), n_vocab = 6000)
  with open('w2v_word2idx.json', 'w') as f:
    json.dump(word2idx, f)

  vocab_sz = len(word2idx)
  model = Word2Vec(80, vocab_sz, 3)
  model.fit(sentences, lr = 1e-4, epochs = 20)
  model.save('w2v_model.npz')

if __name__=='__main__':
  run()